In [2]:
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.schema import Document
import pinecone
from pypdf import PdfReader
#from langchain.llms.openai import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain import HuggingFaceHub

c:\Users\AKM098\AppData\Local\anaconda3\envs\llm-mysql\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [26]:
import glob

In [3]:
#Extract Information from PDF file
def get_pdf_text(pdf_doc):
    text = ""
    pdf_reader = PdfReader(pdf_doc)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [75]:
pdf = "docs/embedded-software-engineer-resume-example.pdf"

In [76]:
pdf_doc = get_pdf_text(pdf)
pdf_doc

'SALVADOR SANZ\nEmbedded Software Engineer\nsalvador.sanz@email.com (123) 456-7890 Dover, DE\nLinkedIn\nWORK EXPERIENCE\nSenior Embedded Software Engineer\nDice\n2019 - current Dover, DE\nContributed to the design of software system architecture\nto establish a stable foundation for 25+ applications\nManaged 3 long-term development programs and 6\nshort-cycle custom design tasks\nOwned 100% of technical leadership responsibilities for\nproduct feature research and development\nSpearheaded all aspects of product development from\nsystem design to testing\nSoftware Engineer III, Embedded Systems\nGoogle\n2015 - 2019 Dover, DE\nArchitected, implemented, and maintained 50+ complex\nﬁrmware/software designs\nWrote clean, robust, testable, reusable, and sustainable\ncode tailored for embedded systems\nWorked with 4 multi-functional teams to determine\nsoftware architecture and speciﬁcations\nDeveloped and maintained 12+ engineering tools used to\ndebug, analyze, and test embedded products\nR

In [72]:
dir = r'c:/Users/AKM098/OneDrive - Maersk Group/Data_Science/End-to-End-implementation/LLMApps/LLMApp-ResumeScreening/docs/'
files = glob.glob(dir+'./*.pdf')
files[:2]

['c:/Users/AKM098/OneDrive - Maersk Group/Data_Science/End-to-End-implementation/LLMApps/LLMApp-ResumeScreening/docs/.\\embedded-software-engineer-resume-example.pdf',
 'c:/Users/AKM098/OneDrive - Maersk Group/Data_Science/End-to-End-implementation/LLMApps/LLMApp-ResumeScreening/docs/.\\java-programmer-resume-example.pdf']

In [73]:
pdf_list = files[:2]

In [4]:
def create_docs(user_pdf_list, unique_id):
    docs=[]
    for filename in user_pdf_list:
        
        chunks=get_pdf_text(filename)

        #Adding items to our list - Adding data & its metadata
        docs.append(Document(
            page_content=chunks,
            metadata={"name": filename.name,"id":filename.id,"type=":filename.type,"size":filename.size,"unique_id":unique_id},
        ))

    return docs

In [81]:
for filename in pdf_list:
    print(filename.name)

AttributeError: 'str' object has no attribute 'name'

In [77]:
 chunks=get_pdf_text(pdf)

In [78]:
chunks

'SALVADOR SANZ\nEmbedded Software Engineer\nsalvador.sanz@email.com (123) 456-7890 Dover, DE\nLinkedIn\nWORK EXPERIENCE\nSenior Embedded Software Engineer\nDice\n2019 - current Dover, DE\nContributed to the design of software system architecture\nto establish a stable foundation for 25+ applications\nManaged 3 long-term development programs and 6\nshort-cycle custom design tasks\nOwned 100% of technical leadership responsibilities for\nproduct feature research and development\nSpearheaded all aspects of product development from\nsystem design to testing\nSoftware Engineer III, Embedded Systems\nGoogle\n2015 - 2019 Dover, DE\nArchitected, implemented, and maintained 50+ complex\nﬁrmware/software designs\nWrote clean, robust, testable, reusable, and sustainable\ncode tailored for embedded systems\nWorked with 4 multi-functional teams to determine\nsoftware architecture and speciﬁcations\nDeveloped and maintained 12+ engineering tools used to\ndebug, analyze, and test embedded products\nR

In [79]:
docs=[]

In [83]:
docs.append(Document(page_content=chunks))

In [84]:
docs

[Document(page_content='SALVADOR SANZ\nEmbedded Software Engineer\nsalvador.sanz@email.com (123) 456-7890 Dover, DE\nLinkedIn\nWORK EXPERIENCE\nSenior Embedded Software Engineer\nDice\n2019 - current Dover, DE\nContributed to the design of software system architecture\nto establish a stable foundation for 25+ applications\nManaged 3 long-term development programs and 6\nshort-cycle custom design tasks\nOwned 100% of technical leadership responsibilities for\nproduct feature research and development\nSpearheaded all aspects of product development from\nsystem design to testing\nSoftware Engineer III, Embedded Systems\nGoogle\n2015 - 2019 Dover, DE\nArchitected, implemented, and maintained 50+ complex\nﬁrmware/software designs\nWrote clean, robust, testable, reusable, and sustainable\ncode tailored for embedded systems\nWorked with 4 multi-functional teams to determine\nsoftware architecture and speciﬁcations\nDeveloped and maintained 12+ engineering tools used to\ndebug, analyze, and te

In [ ]:
docs.

In [5]:
#Create embeddings instance
def create_embeddings_load_data():
    #embeddings = OpenAIEmbeddings()
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    return embeddings

In [85]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\AKM098\AppData\Local\anaconda3\envs\llm-mysql\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [88]:
pinecone_apikey = "e55e23e2-8a3b-4b6a-a2b1-5e3076fd28eb"
os.environ['PINECONE_API_KEY'] = pinecone_apikey
pinecone_index_name='resume'

In [89]:
index = Pinecone.from_documents(docs, embeddings, index_name=pinecone_index_name)

In [90]:
index

In [91]:
index = Pinecone.from_existing_index(pinecone_index_name, embeddings)

In [92]:
index

In [ ]:
index.similarity_search(query, k=2, {"unique_id": unique_id})

In [ ]:
#Function to push data to Pinecone
def push_to_pinecone(pinecone_index_name,embeddings, docs):
    index = Pinecone.from_documents(docs, embeddings, index_name=pinecone_index_name)
    return index

#Function to pull index data from Pinecone
def pull_from_pinecone(pinecone_index_name,embeddings):
    index = Pinecone.from_existing_index(pinecone_index_name, embeddings)
    return index

#This function will help us in fetching the top relevent documents from our vector store - Pinecone Index
def get_similar_docs(index,query,k=2,  unique_id):
    similar_docs = index.similarity_search(query, k=k, {"unique_id": unique_id})
    return similar_docs

In [ ]:
pinecone_index_name

In [ ]:
index = Pinecone.from_documents(docs, embeddings, index_name=pinecone_index_name)

In [9]:
def get_summary(current_doc):
    #llm = OpenAI(temperature=0)
    llm = HuggingFaceHub(repo_id="bigscience/bloom", model_kwargs={"temperature":0})
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    summary = c hain.run([current_doc])

    return summary

In [10]:
import uuid

In [17]:
pdf_doc

'SALVADOR SANZ\nEmbedded Software Engineer\nsalvador.sanz@email.com (123) 456-7890 Dover, DE\nLinkedIn\nWORK EXPERIENCE\nSenior Embedded Software Engineer\nDice\n2019 - current Dover, DE\nContributed to the design of software system architecture\nto establish a stable foundation for 25+ applications\nManaged 3 long-term development programs and 6\nshort-cycle custom design tasks\nOwned 100% of technical leadership responsibilities for\nproduct feature research and development\nSpearheaded all aspects of product development from\nsystem design to testing\nSoftware Engineer III, Embedded Systems\nGoogle\n2015 - 2019 Dover, DE\nArchitected, implemented, and maintained 50+ complex\nﬁrmware/software designs\nWrote clean, robust, testable, reusable, and sustainable\ncode tailored for embedded systems\nWorked with 4 multi-functional teams to determine\nsoftware architecture and speciﬁcations\nDeveloped and maintained 12+ engineering tools used to\ndebug, analyze, and test embedded products\nR

In [13]:
unique_id = uuid.uuid4().hex

In [74]:
final_docs_list = create_docs(pdf_list,unique_id)

AttributeError: 'str' object has no attribute 'name'